In [1]:
import pyranges as pr
import numpy as np
import pandas as pd
import os
import sys
import time 

In [2]:
# A plus strand transcript
test_ref_tr1 = {"Chromosome": [1]*4,
                "Start": [10,100,200,300],
                "End": [30,120,220,340],
                "Strand": ["+"]*4,
                "Feature": ["exon"]*4,
                "gene_id": ["ref_gene_1"]*4,
                "transcript_id": ["ref_tr_1"] * 4}


pr.from_dict(test_ref_tr1)


,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,ref_gene_1,ref_tr_1
1,1,100,120,+,exon,ref_gene_1,ref_tr_1
2,1,200,220,+,exon,ref_gene_1,ref_tr_1
3,1,300,340,+,exon,ref_gene_1,ref_tr_1


In [3]:
{"Chromosome": [],
                "Start": [],
                "End": [],
                "Strand": [],
                "Feature": [],
                "gene_id": [],
                "transcript_id": []
               }

{'Chromosome': [],
 'Start': [],
 'End': [],
 'Strand': [],
 'Feature': [],
 'gene_id': [],
 'transcript_id': []}

In [4]:
# A minus strand transcript
test_ref_tr2 = {"Chromosome": [2]*3,
                "Start": [10,80,100],
                "End": [20,90,120],
                "Strand": ["-"]*3,
                "Feature": ["exon"]*3,
                "gene_id": ["ref_gene_2"]*3,
                "transcript_id": ["ref_tr_2"]*3
               }


test_ref = pr.concat([pr.from_dict(test_ref_tr1), pr.from_dict(test_ref_tr2)])
test_ref

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,ref_gene_1,ref_tr_1
1,1,100,120,+,exon,ref_gene_1,ref_tr_1
2,1,200,220,+,exon,ref_gene_1,ref_tr_1
3,1,300,340,+,exon,ref_gene_1,ref_tr_1
4,2,10,20,-,exon,ref_gene_2,ref_tr_2
5,2,80,90,-,exon,ref_gene_2,ref_tr_2
6,2,100,120,-,exon,ref_gene_2,ref_tr_2


In [5]:
# Now make test novel transcripts to cover my test cases

# 1. Novel last exons in first intron of annotated transcript (e.g. STMN2)
# For these to pass, they should share an identical 3'end with a first exon of a known transcript

test_novel_fi = {"Chromosome": [1]*4,
                "Start": [10,50]*2,
                "End": [30,70,35,70],
                "Strand": ["+"]*4,
                "Feature": ["exon"]*4,
                "gene_id": ["nov_gene_1"]*4,
                "transcript_id": ["nov_tx_fi_p"]*2 + ["nov_tx_fi_f"]*2,
               }

pr.from_dict(test_novel_fi)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_fi_p
1,1,50,70,+,exon,nov_gene_1,nov_tx_fi_p
2,1,10,35,+,exon,nov_gene_1,nov_tx_fi_f
3,1,50,70,+,exon,nov_gene_1,nov_tx_fi_f


In [6]:
# 2. Internal intron, spliced in last exon (fully contained within last exon) (e.g. ONECUT1)
# For these to pass, they should match the intron chain of a known transcript up until the penultimate exon

test_novel_si = {"Chromosome": [1]*6,
                "Start": [10,100,140] + [50,100,140],
                "End": [30,120,160] + [70,120,160],
                "Strand": ["+"]*6,
                "Feature": ["exon"]*6,
                "gene_id": ["nov_gene_1"]*6,
                "transcript_id": ["nov_tx_si_p"]*3 + ["nov_tx_si_f"]*3
               }

pr.from_dict(test_novel_si)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_si_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_si_p
2,1,140,160,+,exon,nov_gene_1,nov_tx_si_p
3,1,50,70,+,exon,nov_gene_1,nov_tx_si_f
4,1,100,120,+,exon,nov_gene_1,nov_tx_si_f
5,1,140,160,+,exon,nov_gene_1,nov_tx_si_f


In [7]:
# 3. Internal intron bleedthrough (e.g. SIN3B)
# For these events to pass, they should match the intron chain of a known transcript up until the penultimate exon

test_novel_bl = {"Chromosome": [1]*6,
                "Start": [10,100,200] + [50,100,200],
                "End": [30,120,240] + [70,120,240],
                "Strand": ["+"]*6,
                "Feature": ["exon"]*6,
                "gene_id": ["nov_gene_1"]*6,
                "transcript_id": ["nov_tx_bl_p"]*3 + ["nov_tx_bl_f"]*3
               }

pr.from_dict(test_novel_bl)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_bl_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_bl_p
2,1,200,240,+,exon,nov_gene_1,nov_tx_bl_p
3,1,50,70,+,exon,nov_gene_1,nov_tx_bl_f
4,1,100,120,+,exon,nov_gene_1,nov_tx_bl_f
5,1,200,240,+,exon,nov_gene_1,nov_tx_bl_f


In [8]:
# 4. Internal intron with novel internal and terminal exon (e.g.)
# For these events to pass, they should match the intron chain of a known transcript,
# but have a continuous chain of length n of novel events at the 3'end of the transcript
# (n can be varied)
# Event know from NP is fully contained within annotated intron - also set this constraint?

test_novel_mult = {"Chromosome": [1]*4,
                   "Start": [10,100,130,150],
                   "End": [30,120,140,160],
                   "Strand": ["+"]*4,
                   "Feature": ["exon"]*4,
                   "gene_id": ["nov_gene_1"]*4,
                   "transcript_id": ["nov_tx_mult_p"]*4
                    }

pr.from_dict(test_novel_mult)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_mult_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_mult_p
2,1,130,140,+,exon,nov_gene_1,nov_tx_mult_p
3,1,150,160,+,exon,nov_gene_1,nov_tx_mult_p


In [9]:
# 5. 3'UTR intron fully contained within an annotated 3'UTR (e.g. TDP-43)
# For this to pass, they should match the intron chain of a known transcript up until the penultimate exon
# (Annotate as a 3'UTR intron (spliced out) after filtering for intron chain match)
test_novel_3ui = {"Chromosome": [1]*5,
                "Start": [10,100,200,300,330],
                "End": [30,120,220,310,340],
                "Strand": ["+"]*5,
                "Feature": ["exon"]*5,
                "gene_id": ["nov_gene_1"]*5,
                "transcript_id": ["nov_tx_3ui_p"]*5
                 }

pr.from_dict(test_novel_3ui)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_3ui_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_3ui_p
2,1,200,220,+,exon,nov_gene_1,nov_tx_3ui_p
3,1,300,310,+,exon,nov_gene_1,nov_tx_3ui_p
4,1,330,340,+,exon,nov_gene_1,nov_tx_3ui_p


In [10]:
#6. Distal last exon spliced from penultimate exon (i.e. a mutually exclusive last exon) (e.g. SMC1A)
# For this to pass, they should match the intron chain of a known transcript up until the penultimate exon
# Will have to annotate more precisely later (i.e. differentiate from 7)

test_novel_exc_dist = {"Chromosome": [1]*4,
                "Start": [10,100,200,360],
                "End": [30,120,220,380],
                "Strand": ["+"]*4,
                "Feature": ["exon"]*4,
                "gene_id": ["nov_gene_1"]*4,
                "transcript_id": ["nov_tx_exc_dist_p"]*4
               }

pr.from_dict(test_novel_exc_dist)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_exc_dist_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_exc_dist_p
2,1,200,220,+,exon,nov_gene_1,nov_tx_exc_dist_p
3,1,360,380,+,exon,nov_gene_1,nov_tx_exc_dist_p


In [11]:
#7. Distal last exon spliced from ann
# For this to pass, they should match the intron chain of a known transcript up until the penultimate exon
test_novel_le_dist = {"Chromosome": [1]*5,
                      "Start": [10,100,200,300,360],
                      "End": [30,120,220,310,380],
                      "Strand": ["+"]*5,
                      "Feature": ["exon"]*5,
                      "gene_id": ["nov_gene_1"]*5,
                      "transcript_id": ["nov_tx_le_dist_p"]*5
                     }

pr.from_dict(test_novel_le_dist)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_le_dist_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_le_dist_p
2,1,200,220,+,exon,nov_gene_1,nov_tx_le_dist_p
3,1,300,310,+,exon,nov_gene_1,nov_tx_le_dist_p
4,1,360,380,+,exon,nov_gene_1,nov_tx_le_dist_p


In [12]:
#8. Distal last exon spliced from annoatrd (minus strand)
# For this to pass, they should match the intron chain of a known transcript up until the penultimate exon
test_novel_le_dist_minus = {"Chromosome": [2]*3,
                      "Start": [40,80,100],
                      "End": [50,90,120],
                      "Strand": ["-"]*3,
                      "Feature": ["exon"]*3,
                      "gene_id": ["nov_gene_2"]*3,
                      "transcript_id": ["nov_tx_le_dist_p_minus"]*3
                     }

pr.from_dict(test_novel_le_dist_minus)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,2,40,50,-,exon,nov_gene_2,nov_tx_le_dist_p_minus
1,2,80,90,-,exon,nov_gene_2,nov_tx_le_dist_p_minus
2,2,100,120,-,exon,nov_gene_2,nov_tx_le_dist_p_minus


In [13]:
test_novel_gr = pr.concat([pr.from_dict(event) for event in [test_novel_3ui,
                                                            test_novel_bl,
                                                            test_novel_exc_dist,
                                                            test_novel_fi,
                                                            test_novel_le_dist,
                                                            test_novel_mult,
                                                            test_novel_si,
                                                            test_novel_le_dist_minus]
                          ]
                         )

test_novel_gr

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,10,30,+,exon,nov_gene_1,nov_tx_3ui_p
1,1,100,120,+,exon,nov_gene_1,nov_tx_3ui_p
2,1,200,220,+,exon,nov_gene_1,nov_tx_3ui_p
3,1,300,310,+,exon,nov_gene_1,nov_tx_3ui_p
4,1,330,340,+,exon,nov_gene_1,nov_tx_3ui_p
5,1,10,30,+,exon,nov_gene_1,nov_tx_bl_p
6,1,100,120,+,exon,nov_gene_1,nov_tx_bl_p
7,1,200,240,+,exon,nov_gene_1,nov_tx_bl_p
8,1,50,70,+,exon,nov_gene_1,nov_tx_bl_f
9,1,100,120,+,exon,nov_gene_1,nov_tx_bl_f


In [14]:
def introns_from_df(df):
    '''
    '''
    
    n_exons = len(df)
    
    if n_exons < 2:
        return None
        #print(df)
        #raise Exception("at least two exons are required for transcript to have an intron")
    # n exons = n-1 introns
    
    strand = df["Strand"].drop_duplicates().tolist()[0]
#     print(strand)
    chrom = df["Chromosome"].drop_duplicates().tolist()[0]
    gene_id = df["gene_id"].drop_duplicates().tolist()[0]
    tx_id = df["transcript_id"].drop_duplicates().tolist()[0]
    feature = "intron"
    introns = {}
    for i in range(0, n_exons - 1):
        if strand == "+":
            intron_start = df.iloc[i, lambda x: x.columns.get_loc("End")]
            intron_end = df.iloc[i+1, lambda x: x.columns.get_loc("Start")]
            introns[str(i)] = {"Chromosome": chrom,
                               "Start": intron_start,
                               "End": intron_end,
                               "Strand": strand,
                               "Feature": feature,
                               "gene_id": gene_id,
                               "transcript_id": tx_id}
        elif strand == "-":
            intron_start = df.iloc[i, lambda x: x.columns.get_loc("End")]
            intron_end = df.iloc[i+1, lambda x: x.columns.get_loc("Start")]
            introns[str(i)] = {"Chromosome": chrom,
                               "Start": intron_start,
                               "End": intron_end,
                               "Strand": strand,
                               "Feature": feature,
                               "gene_id": gene_id,
                               "transcript_id": tx_id}
    return pd.DataFrame.from_dict(introns, orient = "index")
        


def introns_by_tx(gr, by="transcript_id", nb_cpu=1):
    '''
    '''
    
    return gr.apply(lambda df: df.groupby(by).apply(introns_from_df), nb_cpu=1)
    
    
test_novel_introns = introns_by_tx(test_novel_gr)
test_ref_introns = introns_by_tx(test_ref)
    
    
test_novel_introns

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,30,100,+,intron,nov_gene_1,nov_tx_3ui_p
1,1,120,200,+,intron,nov_gene_1,nov_tx_3ui_p
2,1,220,300,+,intron,nov_gene_1,nov_tx_3ui_p
3,1,310,330,+,intron,nov_gene_1,nov_tx_3ui_p
4,1,70,100,+,intron,nov_gene_1,nov_tx_bl_f
5,1,120,200,+,intron,nov_gene_1,nov_tx_bl_f
6,1,30,100,+,intron,nov_gene_1,nov_tx_bl_p
7,1,120,200,+,intron,nov_gene_1,nov_tx_bl_p
8,1,30,100,+,intron,nov_gene_1,nov_tx_exc_dist_p
9,1,120,200,+,intron,nov_gene_1,nov_tx_exc_dist_p


In [15]:
test_ref_introns

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id
0,1,30,100,+,intron,ref_gene_1,ref_tr_1
1,1,120,200,+,intron,ref_gene_1,ref_tr_1
2,1,220,300,+,intron,ref_gene_1,ref_tr_1
3,2,20,80,-,intron,ref_gene_2,ref_tr_2
4,2,90,100,-,intron,ref_gene_2,ref_tr_2


In [16]:
def rle(inarray):
        """
        run length encoding. Partial credit to R rle function. 
        Multi datatype arrays catered for including non Numpy
        returns: tuple (runlengths, startpositions, values)
        https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
        Thomas Browne
        """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [17]:
# Dict of {novel_tx_id: {matches: [ref_id], chain_match: [n_matching], terminal_non_match: [n_not_matching]}}
novel_info_dict = {}

test_novel_gr.apply(lambda df: df.groupby("transcript_id").filter(lambda grp: len(grp) > 2))

for key, dfs in pr.itergrs([test_novel_introns.apply(lambda df: df.groupby("transcript_id").filter(lambda grp: len(grp) > 1)), test_ref_introns], strand=True, keys=True):
    print("----processing chrom and strand pair {0} & {1}".format(key[0], key[1]))
  
    by_tx = tuple([df.groupby("transcript_id") for df in dfs])
    novel_txipts = by_tx[0]
    ref_txipts = by_tx[1]
    
    strand = key[1]
#     print(strand)
#    #1. for each novel transcript, test it against all ref transcripts:
    
    
    for novel_id, novel_introns in novel_txipts:
#         print(novel_id)
#         print(novel_introns)
#         print("class of novel_introns is {}".format(type(novel_introns)))
        #print(exons)
        
        if strand == "-":
            #First in order = last intron - reverse df now
            novel_introns = novel_introns[::-1].reset_index(drop=True)
#             ref_introns = ref_introns[::-1].reset_index(drop=True)
            #print(exons)
        #by_tx[1]
        else:
            pass
        
        
        if strand == "+":
            first_matches = [np.array_equal(novel_introns.head(1)[["Start","End"]],
                                            ref_introns.head(1)[["Start","End"]]) for ref_id,ref_introns in ref_txipts]
        else:
            first_matches = [np.array_equal(novel_introns.head(1)[["Start","End"]],
                                            ref_introns[::-1].reset_index(drop=True) # rev order
                                            .head(1)[["Start","End"]]
                                           )
                             for ref_id,ref_introns in ref_txipts]

        #print(novel_id)
#         print(first_matches)
        if not sum(first_matches) > 0:
            print("{0} does not match any reference transcripts in its first intron. Skipping".format(novel_id))
            continue
        
        # Now compare novel transcript against each ref transcript with match in first intron.
        # 
        for ref_tr, first_match in zip(ref_txipts, first_matches):
            
            if not first_match:
                continue
            #intron_chain_match = np.equal(np.asarray(exons[["Start","End"]]), np.asarray(ref_tr[1][["Start","End"]]))
            ref_introns = ref_tr[1]
            
            if strand == "-":
                ref_introns = ref_introns[::-1].reset_index(drop=True) # rev order
            
            n_novel_introns = len(novel_introns)
            n_ref_introns = len(ref_introns)
            
            # To avoid a slicing error for txipts shorter than novel
            max_chain = min(n_novel_introns, n_ref_introns)
#             print(max_chain)
#             print(ref_tr[1].iloc[0,])
#             print(n_novel_introns)
#             print(len(ref_tr[1]))
            novel_chain_match = pd.DataFrame([(novel_introns.iloc[i,:][["Start","End"]]
                                               .eq(ref_introns.iloc[i,:][["Start","End"]]
                                                  )
                                              )
                                              for i in range(max_chain)])
    
            # Collapse to single True/False per row - does intron match?
            novel_chain_match = novel_chain_match.apply(np.all, axis=1, raw=True)
            
#             print(novel_chain_match)
            
            runs, starts, vals = rle(novel_chain_match)
            #print(runs[0])
            #print(starts)
            #print("\n {0}".format(np.where(vals == False)))
            
            if np.all(vals) or np.array_equal(vals, [True,False]):
                # Possible genuine match, should update dict with info
                
                # match (from start of ref Txipt)
                
                if vals.size == 1:
                    #i.e. all true/introns match (e.g. bleedthrough/ where )
                    terminal_non_match = 0
                else:
                    terminal_non_match = runs[1]
                
#                 print(terminal_non_match)
                
                if novel_id not in novel_info_dict:
                    
                    novel_info_dict[novel_id] = {"matches": [ref_tr[0]],
                                                 "chain_match": [runs[0]], #Always starts with true, so take length of true
                                                 "terminal_non_match": [terminal_non_match],
                                                }
                
                else:
                    # Append to dict
                    novel_info_dict[novel_id]["matches"].append(ref_tr[0])
                    novel_info_dict[novel_id]["chain_match"].append(ref_tr[0])
                    novel_info_dict[novel_id]["terminal_non_match"].append(ref_tr[0])
                    
                
            else:
                continue
            
            
            
print(novel_info_dict)
#             intron_chain_match = exons[["Start","End"]].eq(ref_tr[1][["Start","End"]])
#             print("\n")
#             print(intron_chain_match)
#             print(intron_chain_match.apply(np.all, axis = 1))
            
            
                #loc[np.where(lambda x: (x[["Start", "End"]].all()))])
                                                  #apply(lambda x: all(x), axis = "index", raw = True))
#             print(intron_chain_match)
            
            
#         first_intron = exons.head(1)[["Start","End"]] if strand == "+" else exons.tail(1)[["Start","End"]]
        
#         grp_filter_matches = by_tx[1].filter(lambda x: np.array_equal(first_intron, x.head(1)[["Start", "End"]]))
#         print(grp_filter_matches)
#         print(type(grp_filter_matches))
     #for novel, ref_tr in zip(by_tx[0],by_tx[1]:
#         print(nov_tr[1])
#         print(ref_tr)
             
pd.concat({key: pd.DataFrame.from_dict(d, orient = "columns") for key, d in novel_info_dict.items()}, axis=0)

----processing chrom and strand pair 1 & +
nov_tx_bl_f does not match any reference transcripts in its first intron. Skipping
nov_tx_si_f does not match any reference transcripts in its first intron. Skipping
----processing chrom and strand pair 2 & -
{'nov_tx_3ui_p': {'matches': ['ref_tr_1'], 'chain_match': [3], 'terminal_non_match': [0]}, 'nov_tx_bl_p': {'matches': ['ref_tr_1'], 'chain_match': [2], 'terminal_non_match': [0]}, 'nov_tx_exc_dist_p': {'matches': ['ref_tr_1'], 'chain_match': [2], 'terminal_non_match': [1]}, 'nov_tx_le_dist_p': {'matches': ['ref_tr_1'], 'chain_match': [3], 'terminal_non_match': [0]}, 'nov_tx_mult_p': {'matches': ['ref_tr_1'], 'chain_match': [1], 'terminal_non_match': [2]}, 'nov_tx_si_p': {'matches': ['ref_tr_1'], 'chain_match': [1], 'terminal_non_match': [1]}, 'nov_tx_le_dist_p_minus': {'matches': ['ref_tr_2'], 'chain_match': [1], 'terminal_non_match': [1]}}


,,matches,chain_match,terminal_non_match
nov_tx_3ui_p,0,ref_tr_1,3,0
nov_tx_bl_p,0,ref_tr_1,2,0
nov_tx_exc_dist_p,0,ref_tr_1,2,1
nov_tx_le_dist_p,0,ref_tr_1,3,0
nov_tx_mult_p,0,ref_tr_1,1,2
nov_tx_si_p,0,ref_tr_1,1,1
nov_tx_le_dist_p_minus,0,ref_tr_2,1,1


In [29]:
def match_intron_chains(novel_gr, ref_gr, id_col = "transcript_id", nb_cpu = 1):
    '''
    grs should contain introns
    '''
    # {novel_id: {matches: [ref_id], chain_match: [n], terminal_non_match: [n]}}
    match_info_dict = {}
    
    for key, dfs in pr.itergrs([novel_gr, ref_gr], strand=True, keys=True):
        print("----processing chrom and strand pair {0} & {1}".format(key[0], key[1]))
        # dfs = novel_gr & ref_gr matched by key (chromosome & strand)
        # pandas group by objects of (transcript_id, df)
        by_tx = tuple([df.groupby("transcript_id") for df in dfs])
        
        novel_txipts = by_tx[0]
        ref_txipts = by_tx[1]
        
        # Pyranges keys are tuples of (chr,strand)
        strand = key[1]
        
        
        #Comparing each novel transcript against all ref transcripts
        for novel_id, novel_introns in novel_txipts:
        
            if strand == "-":
                # Standard PyRanges sort - First in df = last intron (smallest values i.e. leftmost)
                # Reverse so first row in df is always the first intron
                # is reset_index necessary?
                novel_introns = novel_introns[::-1].reset_index(drop=True)
            
            else:
                pass
        
            # As a first pass, check for matches in intron chain of first introns between novel and reference Txs
            if strand == "+":
                first_matches = [np.array_equal(novel_introns.head(1)[["Start","End"]],
                                                ref_introns.head(1)[["Start","End"]]) 
                                 for ref_id,ref_introns in ref_txipts]
            else:
                first_matches = [np.array_equal(novel_introns.head(1)[["Start","End"]],
                                                ref_introns[::-1].reset_index(drop=True) # rev order so 1st row = 1st intron
                                                .head(1)[["Start","End"]]
                                               )
                                 for ref_id,ref_introns in ref_txipts]


            if not sum(first_matches) > 0:
                print("{0} does not match any reference transcripts in its first intron. Skipping".format(novel_id))
                continue
        
            # Compare full introns chains of novel transcript against each ref transcript with match in first intron.
        
            for ref_tr, first_match in zip(ref_txipts, first_matches):
            
                if not first_match:
                    continue
                
            
                ref_id = ref_tr[0]
                print("ref_id {}".format(ref_id))
                ref_introns = ref_tr[1]
            
                if strand == "-":
                    ref_introns = ref_introns[::-1].reset_index(drop=True) # reverse so first row = first intron
                        
                # To avoid a slicing error for ref txipts shorter than novel
                n_novel_introns = len(novel_introns)
                n_ref_introns = len(ref_introns)
                max_chain = min(n_novel_introns, n_ref_introns)
#             print(max_chain)
#             print(ref_tr[1].iloc[0,])
#             print(n_novel_introns)
#             print(len(ref_tr[1]))

                # Row-wise, check whether match with corresponding intron of reference transcript
                novel_chain_match = pd.DataFrame([(novel_introns.iloc[i,:][["Start","End"]]
                                                   .eq(ref_introns.iloc[i,:][["Start","End"]]
                                                      )
                                                  )
                                                  for i in range(max_chain)])
    
                # Collapse to single True/False per row - does intron completely match?
                novel_chain_match = novel_chain_match.apply(np.all, axis=1, raw=True)
            
#             print(novel_chain_match)
            
                runs, starts, vals = rle(novel_chain_match)
            #print(runs[0])
            #print(starts)
            #print("\n {0}".format(np.where(vals == False)))
            
                # Considered a valid match if a intron chain completely identical 
                # or matches at beginning of txipt but differs at the end
                if np.all(vals) or np.array_equal(vals, [True,False]):
                
                    # Don't want to throw away all valid matches (yet)
                    # Possible genuine match, should update dict with info
                    # match (from start of ref Txipt)
                
                    if vals.size == 1:
                        #i.e. all true/introns match (e.g. bleedthrough)
                        terminal_non_match = 0
                    else:
                        # All valid = consective match & non-match, so non-match = 2nd in array 
                        terminal_non_match = runs[1]
                
#                 print(terminal_non_match)
                
                    if novel_id not in match_info_dict:
                    
                        match_info_dict[novel_id] = {"matches": [ref_id],
                                                     "chain_match": [runs[0]], #Always starts with true, so take length of true
                                                     "terminal_non_match": [terminal_non_match],
                                                    }
                
                    else:
                        # Append to dict
                        match_info_dict[novel_id]["matches"].append(ref_id)
                        match_info_dict[novel_id]["chain_match"].append(runs[0])
                        match_info_dict[novel_id]["terminal_non_match"].append(terminal_non_match)
                    
                
                else:
                    continue
    
    # Output df for easier parsing
#     match_df = pd.concat({key: pd.DataFrame.from_dict(d, orient = "columns") for key, d in match_info_dict.items()}, axis=0)
    
    return match_info_dict
        


In [30]:
match_intron_chains(test_novel_introns, test_ref_introns)

----processing chrom and strand pair 1 & +
ref_id ref_tr_1
nov_tx_bl_f does not match any reference transcripts in its first intron. Skipping
ref_id ref_tr_1
ref_id ref_tr_1
nov_tx_fi_f does not match any reference transcripts in its first intron. Skipping
nov_tx_fi_p does not match any reference transcripts in its first intron. Skipping
ref_id ref_tr_1
ref_id ref_tr_1
nov_tx_si_f does not match any reference transcripts in its first intron. Skipping
ref_id ref_tr_1
----processing chrom and strand pair 2 & -
ref_id ref_tr_2


{'nov_tx_3ui_p': {'matches': ['ref_tr_1'],
  'chain_match': [3],
  'terminal_non_match': [0]},
 'nov_tx_bl_p': {'matches': ['ref_tr_1'],
  'chain_match': [2],
  'terminal_non_match': [0]},
 'nov_tx_exc_dist_p': {'matches': ['ref_tr_1'],
  'chain_match': [2],
  'terminal_non_match': [1]},
 'nov_tx_le_dist_p': {'matches': ['ref_tr_1'],
  'chain_match': [3],
  'terminal_non_match': [0]},
 'nov_tx_mult_p': {'matches': ['ref_tr_1'],
  'chain_match': [1],
  'terminal_non_match': [2]},
 'nov_tx_si_p': {'matches': ['ref_tr_1'],
  'chain_match': [1],
  'terminal_non_match': [1]},
 'nov_tx_le_dist_p_minus': {'matches': ['ref_tr_2'],
  'chain_match': [1],
  'terminal_non_match': [1]}}

In [ ]:
np.array_equal(np.array([True, False]), [True, False])
np.all([True,True])

In [ ]:
test_introns = pr.from_dict(test_novel_3ui)#.subset(lambda df: df["transcript_id"] == "nov_tx_3ui_p")

n_exons = len(test_introns)
# n exons = n -1 introns

# plus strand txipt - End of first exon = start of intron, Start of 3'exon = end coord

test_introns.as_df().iloc[0,lambda x: x.columns.get_loc("End")]

def introns_from_df(df):
    '''
    '''
    
    n_exons = len(df)
    
    if n_exons < 2:
        raise Exception("at least two exons are required for transcript to have an intron")
    # n exons = n-1 introns
    
    strand = df["Strand"].drop_duplicates().tolist()[0]
#     print(strand)
    chrom = df["Chromosome"].drop_duplicates().tolist()[0]
    gene_id = df["gene_id"].drop_duplicates().tolist()[0]
    tx_id = df["transcript_id"].drop_duplicates().tolist()[0]
    feature = "intron"
    introns = {}
    for i in range(0, n_exons - 1):
        if strand == "+":
            intron_start = df.iloc[i, lambda x: x.columns.get_loc("End")]
            intron_end = df.iloc[i+1, lambda x: x.columns.get_loc("Start")]
            introns[str(i)] = {"Chromosome": chrom,
                               "Start": intron_start,
                               "End": intron_end,
                               "Strand": strand,
                               "Feature": feature,
                               "gene_id": gene_id,
                               "transcript_id": tx_id}
        elif strand == "-":
            intron_end = df.iloc[i, lambda x: x.columns.get_loc("Start")]
            intron_start = df.iloc[i+1, lambda x: x.columns.get_loc("End")]
            introns[str(i)] = {"Chromosome": chrom,
                               "Start": intron_start,
                               "End": intron_end,
                               "Strand": strand,
                               "Feature": feature,
                               "gene_id": gene_id,
                               "transcript_id": tx_id}
    return pd.DataFrame.from_dict(introns, orient = "index")
        


def introns_by_tx(gr, by="transcript_id", nb_cpu=1):
    '''
    '''
    
    return gr.apply(lambda df: df.groupby(by).apply(introns_from_df), nb_cpu=1)
    
    
test_introns

In [ ]:
introns_by_tx(test_introns)

In [ ]:
introns_by_tx(pr.data.ensembl_gtf().subset(lambda df: (df.gene_id == "ENSG00000205231") & (df.Feature == "exon")))

In [ ]:

pr.data.ensembl_gtf().subset(lambda df: df.transcript_id == "ENST00000450305")[["transcript_id","Feature"]].features.introns(by="transcript").dtypes

In [ ]:
# test_ref.features.introns(by="transcript")
(pr.data.ensembl_gtf().subset(lambda df: df.transcript_id == "ENST00000450305")
#  [["gene_id","transcript_id"]]
 .features.introns(by="transcript"))

In [ ]:
pr.data.ensembl_gtf().subset(lambda df: (df.gene_id == "ENSG00000205231") & (df.Feature == "exon"))

In [ ]:
pr.data.ensembl_gtf().subset(lambda df: (df.gene_id == "ENSG00000205231")).features.introns(by="transcript")
#Start of first feature, end of next along (if properly sorted)

In [ ]:
path_stie_gtf_chr1_nov = "../two_sample_example_output/stringtie/"
path_ref_gtf = ""